In [1]:
import numpy as np
from keras.utils import np_utils
import os, cv2
os.chdir("../")
os.getcwd()

'c:\\Users\\hp\\Documents\\iNeuron_intern\\covid_mask'

In [2]:
from covid_md.constant import *
from covid_md.utils import read_yaml, create_directories

In [13]:
from pathlib import Path
from dataclasses import dataclass
@dataclass(frozen= True)
class DataPreprocessingConfig:
    root_dir: Path
    data_path: Path
    preprocess_records: Path

In [30]:
class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def data_preprocessing(self)->DataPreprocessingConfig:
        config= self.config.data_preprocessing
        create_directories([config.root_dir])
        data_preprocessing_config= DataPreprocessingConfig(
            root_dir= self.config.data_preprocessing.root_dir,
            data_path= self.config.data_preprocessing.data_path,
            preprocess_records= self.config.data_preprocessing.preprocess_records
        )
        return data_preprocessing_config

In [31]:
class DataPreprocessing:
    def __init__(self, config: DataPreprocessingConfig):
        self.config= config
    def data_prep(self):
        img_size= 100
        data= []
        target= []
        path= self.config.data_path
        categories= os.listdir(path)
        labels= [i for i in range(len(categories))]
        labels_dict= dict(zip(categories, labels))
        for category in categories:
            folder_path= os.path.join(self.config.data_path, category)
            img_names= os.listdir(folder_path)
            for img_name in img_names:
                img_path= os.path.join(folder_path, img_name)
                img= cv2.imread(img_path)
                try:
                    gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    resized= cv2.resize(gray,(img_size,img_size))
                    data.append(resized)
                    target.append(labels_dict[category])
                except Exception as e:
                    raise e
        os.chdir(self.config.preprocess_records)
        data= np.array(data)/255
        data= np.reshape(data,(data.shape[0],img_size,img_size,1))
        target= np.array(target)
        new_target= np_utils.to_categorical(target)
        np.save('data', data)
        np.save('target',new_target)

In [32]:
try:
    config= ConfigurationManager()
    data_preprocessing_config= config.data_preprocessing()
    data_preprocessing= DataPreprocessing(config= data_preprocessing_config)
    data_preprocessing.data_prep()
except Exception as e:
    raise e

In [33]:
os.getcwd()

'c:\\Users\\hp\\Documents\\iNeuron_intern\\covid_mask\\artifacts\\data_preprocess'